In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../data/raw/movies_dataset_2000_2023.csv')
#df = pd.read_csv('movies_dataset_2000_2023.csv')

In [3]:
df.shape

(6095, 24)

In [4]:
df.columns

Index(['Unnamed: 0.1', 'Movie Title', 'Plot Summary', 'Cover Image', 'Year',
       'Source', 'Rated', 'omdb Year', 'omdb Title', 'Released', 'Runtime',
       'Genre', 'Director', 'Writer', 'Actors', 'Plot', 'Language', 'Country',
       'Awards', 'Poster', 'Ratings', 'imdbRating', 'BoxOffice', 'imdbID'],
      dtype='object')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6095 entries, 0 to 6094
Data columns (total 24 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0.1  896 non-null    float64
 1   Movie Title   6095 non-null   object 
 2   Plot Summary  5556 non-null   object 
 3   Cover Image   5990 non-null   object 
 4   Year          6095 non-null   int64  
 5   Source        6095 non-null   object 
 6   Rated         841 non-null    object 
 7   omdb Year     871 non-null    object 
 8   omdb Title    871 non-null    object 
 9   Released      868 non-null    object 
 10  Runtime       861 non-null    object 
 11  Genre         870 non-null    object 
 12  Director      850 non-null    object 
 13  Writer        857 non-null    object 
 14  Actors        867 non-null    object 
 15  Plot          862 non-null    object 
 16  Language      866 non-null    object 
 17  Country       869 non-null    object 
 18  Awards        756 non-null  

In [6]:
!pip install nltk

In [7]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/shreyabaral/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/shreyabaral/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [8]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
# import string library for text preprocessing
import string

from nltk.stem import WordNetLemmatizer

nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/shreyabaral/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [9]:
# Convert text to lowercase
def convert_to_lower_case(text):
  return text.lower()


In [10]:
def remove_punctuation(text):
  # Create a translation table to remove punctuation
  translator = str.maketrans('', '', string.punctuation)

  # Remove punctuation using the translation table
  text_without_punct = text.translate(translator)

  return text_without_punct


In [11]:
def tokenize(text):
  # Tokenization
  return word_tokenize(text)

In [12]:
# Remove stopwords
def remove_stop_words(tokens):
  stop_words = set(stopwords.words('english'))
  return [word for word in tokens if word not in stop_words]

In [13]:
def lemmatize(tokens):
  # Lemmatization
  lemmatizer = WordNetLemmatizer()
  return [lemmatizer.lemmatize(word) if lemmatizer.lemmatize(word) is not None else word for word in tokens]



In [14]:
# Remove special characters and numbers
def remove_special_chars(tokens):
  return [re.sub('[^A-Za-z]+', '', word) for word in tokens]

In [15]:
# function to do preprocessing text
def preprocess_text(text):
  lower_text = convert_to_lower_case(text) # converting to lowercase
  removed_punctuation = remove_punctuation(lower_text) # removing punctuation
  tokens = tokenize(removed_punctuation) # tokenize words
  tokens = remove_stop_words(tokens) # removing stop words
  tokens = lemmatize(tokens) # lemmatize tokens
  tokens = remove_special_chars(tokens) # remove special characters from token

  preprocessed_text = ' '.join(tokens) # final preprocessed text
  return preprocessed_text

In [16]:
'''\
def preprocess_text(text):
  # Remove HTML tags and special characters
    clean_text = re.sub('<.*?>', '', text)
    clean_text = re.sub(r'[^a-zA-Z\s]', '', clean_text)

    # Tokenization
    tokens = word_tokenize(clean_text)

    # Remove stop words and convert to lowercase
    stop_words = set(stopwords.words('english'))
    tokens = [token.lower() for token in tokens if token.lower() not in stop_words]

    return ' '.join(tokens)
    '''

"def preprocess_text(text):\n  # Remove HTML tags and special characters\n    clean_text = re.sub('<.*?>', '', text)\n    clean_text = re.sub(r'[^a-zA-Z\\s]', '', clean_text)\n\n    # Tokenization\n    tokens = word_tokenize(clean_text)\n\n    # Remove stop words and convert to lowercase\n    stop_words = set(stopwords.words('english'))\n    tokens = [token.lower() for token in tokens if token.lower() not in stop_words]\n\n    return ' '.join(tokens)\n    "

In [17]:
df = df.dropna(subset=['Plot Summary'])

In [18]:
df['Plot Summary'].isna().sum()

0

In [19]:
# Apply function on text column
df['ProcessedPlot'] = df['Plot Summary'].apply(preprocess_text)

In [20]:
df.tail()

,Unnamed: 0.1,Movie Title,Plot Summary,Cover Image,Year,Source,Rated,omdb Year,omdb Title,Released,...,Plot,Language,Country,Awards,Poster,Ratings,imdbRating,BoxOffice,imdbID,ProcessedPlot
6090,NaN,Memory (2023 film),"Sylvia, a single mother, social worker, and re...",upload.wikimedia.org/wikipedia/en/thumb/4/4e/M...,2023,https://en.wikipedia.org//wiki/Memory_(2023_film),NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sylvia single mother social worker recovering ...
6091,NaN,The Color Purple (2023 film),"In 1909 Georgia, teenager Celie Harris lives w...",upload.wikimedia.org/wikipedia/en/thumb/7/70/C...,2023,https://en.wikipedia.org//wiki/The_Color_Purpl...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,georgia teenager celie harris life sister net...
6092,NaN,The Boys in the Boat (film),The film's plot centres on the University of W...,upload.wikimedia.org/wikipedia/en/thumb/6/63/B...,2023,https://en.wikipedia.org//wiki/The_Boys_in_the...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,film plot centre university washington crew re...
6093,NaN,Ferrari (2023 film),"In the summer of 1957, Italian entrepreneur En...",upload.wikimedia.org/wikipedia/en/thumb/f/f6/F...,2023,https://en.wikipedia.org//wiki/Ferrari_(2023_f...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,summer italian entrepreneur enzo ferrari prep...
6094,NaN,Good Grief (film),"In London, Marc enjoys a Christmas party with ...",upload.wikimedia.org/wikipedia/en/thumb/1/14/G...,2023,https://en.wikipedia.org//wiki/Good_Grief_(film),NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,london marc enjoys christmas party friend oliv...


In [21]:
# Save the preprocessed data
preprocessed_path = '../data/processed/preprocessed_dataset.csv'
df.to_csv(preprocessed_path, index=False)

In [25]:
!pip install textattack


  Obtaining dependency information for textattack from https://files.pythonhosted.org/packages/69/85/f7878f69021c4f6583e07e285380d88f0bf2fafcef32c91dddd4db573692/textattack-0.3.9-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 5.8 MB/s eta 0:00:00
  Obtaining dependency information for editdistance from https://files.pythonhosted.org/packages/b7/a3/058d823b6285c3511dc94ed80620c3fb0c18b4aaa708f70ba71f3af28436/editdistance-0.8.1-cp311-cp311-macosx_11_0_arm64.whl.metadata
  Obtaining dependency information for flair from https://files.pythonhosted.org/packages/af/16/536683088c7306bc51cc3cc58605759ebd83b3f7ffd05a9399f4b99c8614/flair-0.13.1-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 769.7/769.7 kB 5.9 MB/s eta 0:00:0000:0100:01
  Obtaining dependency information for lru-dict from https://files.pythonhosted.org/packages/4e/63/a0ae20525f9d52f62ac0def47935f8a2b3b6fcd2c145218b9a27fc1fb910/lru_dict-1.3.0-cp311-cp311-macosx_11_0_arm

  Obtaining dependency information for conllu>=4.0 from https://files.pythonhosted.org/packages/ce/3f/70a1dc5bc536755ec082b806594598a10cfffaf0de978f51d4e0e4fdfa47/conllu-4.5.3-py2.py3-none-any.whl.metadata
  Obtaining dependency information for deprecated>=1.2.13 from https://files.pythonhosted.org/packages/20/8d/778b7d51b981a96554f29136cd59ca7880bf58094338085bcf2a979a0e6a/Deprecated-1.2.14-py2.py3-none-any.whl.metadata
  Obtaining dependency information for gdown>=4.4.0 from https://files.pythonhosted.org/packages/cb/56/f4845ed78723a4eb8eb22bcfcb46e1157a462c78c0a5ed318c68c98f9a79/gdown-5.1.0-py3-none-any.whl.metadata
  Obtaining dependency information for janome>=0.4.2 from https://files.pythonhosted.org/packages/73/7d/70f4069f4bbf0fca023e82a1fbbade6f5216365d4fe259fee1950723eca5/Janome-0.5.0-py2.py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 7.9 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Obtaining dependency information

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.1/67.1 kB 5.3 MB/s eta 0:00:00
  Obtaining dependency information for simpful from https://files.pythonhosted.org/packages/8d/93/8448d3f1aa9d2911b8cba2602aaa1af85eb31a26d28b7b737f1fa5b40c02/simpful-2.11.1-py3-none-any.whl.metadata
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 436.8/436.8 kB 10.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.3/388.3 kB 8.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.3/143.3 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 8.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.6/202.6

  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993225 sha256=fbdcc3902c2fd86eee848a69f04986f41470da8fcf3b03e8faf9ff6dfb2e7449
  Stored in directory: /Users/shreyabaral/Library/Caches/pip/wheels/0a/f2/b2/e5ca405801e05eb7c8ed5b3b4bcf1fcabcd6272c167640072e
  Created wheel for pptree: filename=pptree-3.1-py3-none-any.whl size=4608 sha256=f0181d684129646256a57e0d7c6e1063d44932f19aa5a8fc1a92e7e81092a560
  Stored in directory: /Users/shreyabaral/Library/Caches/pip/wheels/68/8a/eb/d683aa6d09dc68ebfde2f37566ddc8807837c4415b4fd2b04c
  Created wheel for sqlitedict: filename=sqlitedict-2.1.0-py3-none-any.whl size=16864 sha256=a3e29bddc3fcf335282babbafc83d3a7503e768acf4af73a8ad460d5a5797cdf
  Stored in directory: /Users/shreyabaral/Library/Caches/pip/wheels/73/63/89/7210274f9b7fb033b8f22671f64c0e0b55083d30c3c046a3ff
  Created wheel for fst-pso: filename=fst_pso-1.8.1-py3-none-any.whl size=20430 sha256=0fedf67c3562c06ffaa557d1ea4352e1636a37c97a68db0f5f08205a23aa04c4


In [28]:
from textattack.augmentation import Augmenter
import pandas as pd

# Example data
data = {'plot': ["This is a positive example.",
                 "I don't like this movie; it's too slow.",
                 "The plot is intriguing and captivating."]}
df = pd.DataFrame(data)

# Function to apply TextAttack augmentation
def augment_text_with_textattack(text):
    augmenter = Augmenter('wordnet')
    augmented_text = augmenter.augment(text)
    return augmented_text

# Apply augmentation to the 'plot' column
df['augmented_plot'] = df['plot'].apply(augment_text_with_textattack)

# Display the results
print(df)

TypeError: 'str' object is not callable